In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load a pre-trained model without the top layers
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model
base_model.trainable = False

# Add our own layers on top
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(train_generator.num_classes, activation='softmax') 
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,       # Random rotation
    width_shift_range=0.1,   # Random horizontal shift
    height_shift_range=0.1,  # Random vertical shift
    shear_range=0.2,         # Shear transformation
    zoom_range=0.2,          # Zoom
    horizontal_flip=True,    # Flip images horizontally
    fill_mode='nearest'      # Filling strategy for new pixels
)


train_generator = train_datagen.flow_from_directory(
    'D:\Parts_reg\heavy_machinery_parts_dataset',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    'D:\Parts_reg\heavy_machinery_parts_dataset',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 354 images belonging to 10 classes.
Found 82 images belonging to 10 classes.


In [10]:
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10
)



d:\Parts_reg\venv\lib\site-packages\PIL\Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 29s 2s/step - accuracy: 0.1840 - loss: 2.2491 - val_accuracy: 0.5488 - val_loss: 1.4674
Epoch 2/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - accuracy: 0.5670 - loss: 1.3472 - val_accuracy: 0.7317 - val_loss: 0.9705
Epoch 3/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 0.7152 - loss: 0.8770 - val_accuracy: 0.7805 - val_loss: 0.7837
Epoch 4/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - accuracy: 0.8337 - loss: 0.5765 - val_accuracy: 0.7561 - val_loss: 0.6823
Epoch 5/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 0.8668 - loss: 0.4551 - val_accuracy: 0.8171 - val_loss: 0.6354
Epoch 6/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - accuracy: 0.9348 - loss: 0.3366 - val_accuracy: 0.7683 - val_loss: 0.5995
Epoch 7/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.9393 - loss: 0.2584 - val_accuracy: 0.7927 - val_loss: 0.5903
Epoch 8/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - accuracy: 0.9024 - loss: 0.3168 - val_accuracy: 0.8171 - val_loss:

In [11]:
model.save('heavy_machine_parts_model.h5')


In [9]:
model = tf.keras.models.load_model('heavy_machine_parts_model.h5')

In [16]:
from tensorflow.keras.preprocessing import image
import numpy as np
import pandas as pd

# Load your parts database
parts_df = pd.read_csv("parts_detail.csv")  # make sure your CSV path is correct

# Load label names
labels = list(train_generator.class_indices.keys())

# Mapping from label index to part_id
label_to_part_id = {
    0: 1001,  # axle_heavy_equipment
    1: 1002,  # boom_arm_excavator
    2: 1003,  # bulldozer_blade
    3: 1004,  # crane_hook
    4: 1005,  # excavator_bucket
    5: 1006,  # gearbox_heavy_machinery
    6: 1007,  # heavy_machinery_engine
    7: 1008,  # hydraulic_cylinder
    8: 1009,  # loader_tire
    9: 1010,  # undercarriage_bulldozer
}

# Load and preprocess the image
img = image.load_img("D:/Parts_reg/test/hc/test6.jpg", target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.

# Predict
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions)
predicted_label = labels[predicted_class]

# Map to part_id
predicted_part_id = label_to_part_id[predicted_class]

# Fetch part details from the parts database
part_details = parts_df[parts_df["part_id"] == predicted_part_id]

# Output
if not part_details.empty:
    part_info = part_details.iloc[0]
    print(f"Predicted Part: {predicted_label}")
    print(f"Part ID: {part_info['part_id']}")
    print(f"Part Name: {part_info['part_name']}")
    print(f"Description: {part_info['description']}")
    print(f"Manufacturer: {part_info['manufacturer']}")
    print(f"Price: ${part_info['price (USD)']}")
    print(f"Weight: {part_info['weight (kg)']} kg")
else:
    print("Part details not found in database.")



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
Predicted Part: heavy_machinery_engine
Part ID: 1007
Part Name: Heavy Machinery Engine
Description: Diesel engine for heavy-duty machines
Manufacturer: Caterpillar
Price: $15000
Weight: 2200 kg


In [17]:
print(train_generator.class_indices)


{'axle_heavy_equipment': 0, 'boom_arm_excavator': 1, 'bulldozer_blade': 2, 'crane_hook': 3, 'excavator_bucket': 4, 'gearbox_heavy_machinery': 5, 'heavy_machinery_engine': 6, 'hydraulic_cylinder': 7, 'loader_tire': 8, 'undercarriage_bulldozer': 9}


In [2]:
import pandas as pd

# Load parts database
parts_df = pd.read_csv('parts_detail.csv')

# Simulate model prediction
predicted_class = 2  # (example: model output)
predicted_part = parts_df.loc[parts_df['part_id'] == predicted_class]

# Display the part details
if not predicted_part.empty:
    print("🔎 Predicted Part Details:")
    print(f"Part Name: {predicted_part.iloc[0]['part_name']}")
    print(f"Part Number: {predicted_part.iloc[0]['part_number']}")
    print(f"Price: {predicted_part.iloc[0]['price']}")
    print(f"Supplier: {predicted_part.iloc[0]['supplier']}")
    print(f"Description: {predicted_part.iloc[0]['description']}")
else:
    print("Part not found!")


🔎 Predicted Part Details:
Part Name: Engine
Part Number: EN-003
Price: 9500
Supplier: Titan Motors
Description: High torque diesel engine


In [18]:
!pip install streamlit



You should consider upgrading via the 'd:\parts_reg\venv\scripts\python.exe -m pip install --upgrade pip' command.
